In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix
import umap
from sklearn.preprocessing import StandardScaler
import random
from scipy.stats.stats import pearsonr
random.seed(123)
from joblib import Parallel, delayed

/tmp/ipykernel_98380/377559581.py:15: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
adata=sc.read_h5ad('/home/jupyter/reference_generating_new/All_consturction/AIFI_Reference_raw.h5ad')

In [15]:
selected_col=['barcodes','subject.biologicalSex', 'subject.ethnicity', 'subject.partnerCode', 
              'subject.race', 'subject.subjectGuid', 'cohort.cohortGuid', 'sample.visitName', 
              'sample.visitDetails', 'subject.birthYear', 'CMV.IgG.Serology.Result.Interpretation',
              'BMI', 'predicted.celltype.l1.score', 'predicted.celltype.l1', 'predicted.celltype.l2.score', 
              'predicted.celltype.l2', 'predicted.celltype.l3.score', 'predicted.celltype.l3', 
              'predicted.celltype.l2.5.score', 'predicted.celltype.l2.5', 'predicted_labels_celltypist', 
              'majority_voting_celltypist', 'doublet_score', 'n_genes_by_counts', 
              'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes',
              'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes',
              'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'leiden', 'leiden_resolution_1', 
              'leiden_resolution_1.5', 'leiden_resolution_2', 'AIFI_L1', 'AIFI_L2', 'AIFI_L3', 'AIFI_L3.5']

In [22]:
adata.obs[selected_col].to_csv("Reference_Label.csv")

In [3]:
import csv
nested_dict={'B cell': {'Effector B cell': ['CD27+ effector B cell',
   'CD27- effector B cell'],
  'Memory B cell': ['Core memory B cell',
   'Early memory B cell',
   'Type 2 polarized memory B cell',
   'CD95 memory B cell',
   'Activated memory B cell'],
  'Naive B cell': ['Core naive B cell', 'ISG+ naive B cell'],
  'Plasma cell': ['Plasma cell'],
  'Transitional B cell': ['Transitional B cell']},
'DC': {'ASDC': ['ASDC'],
  'cDC1': ['cDC1'],
  'cDC2': ['CD14+ cDC2', 'HLA-DRhi cDC2', 'ISG+ cDC2'],
  'pDC': ['pDC']},
'Erythrocyte': {'Erythrocyte': ['Erythrocyte']},
'ILC': {'ILC': ['ILC']},
'Monocyte': {'CD14 monocyte': ['Core CD14 monocyte',
   'ISG+ CD14 monocyte',
   'IL1B+ CD14 monocyte'],
  'CD16 monocyte': ['Core CD16 monocyte',
   'ISG+ CD16 monocyte',
   'C1Q+ CD16 monocyte'],
  'Intermediate monocyte': ['Intermediate monocyte']},
'NK cell': {'CD56bright NK cell': ['CD56bright NK cell'],
  'CD56dim NK cell': ['Adaptive NK cell',
   'GZMK+ CD56dim NK cell',
   'GZMK- CD56dim NK cell',
   'ISG+ CD56dim NK cell'],
  'Proliferating NK cell': ['Proliferating NK cell']},
'Platelet': {'Platelet': ['Platelet']},
'Progenitor cell': {'Progenitor cell': ['CMP cell',
   'CLP cell',
   'BaEoMaP cell']},
'T cell': {'CD8aa': ['CD8aa'],
  'DN T cell': ['DN T cell'],
  'MAIT': ['CD8 MAIT', 'CD4 MAIT', 'ISG+ MAIT'],
  'Memory CD4 T cell': ['CM CD4 T cell',
   'GZMB- CD27+ EM CD4 T cell',
   'GZMB- CD27- EM CD4 T cell',
   'ISG+ memory CD4 T cell',
   'KLRF1- GZMB+ CD27- memory CD4 T cell'],
  'Memory CD8 T cell': ['GZMK+ CD27+ EM CD8 T cell',
   'CM CD8 T cell',
   'KLRF1- GZMB+ CD27- EM CD8 T cell',
   'GZMK- CD27+ EM CD8 T cell',
   'KLRF1+ GZMB+ CD27- EM CD8 T cell',
   'ISG+ memory CD8 T cell'],
  'Naive CD4 T cell': ['Core naive CD4 T cell',
   'SOX4+ naive CD4 T cell',
   'ISG+ naive CD4 T cell'],
  'Naive CD8 T cell': ['Core naive CD8 T cell ',
   'SOX4+ naive CD8 T cell',
   'ISG+ naive CD8 T cell'],
  'Proliferating T cell': ['Proliferating T cell'],
  'Treg': ['Naive CD4 Treg',
   'Memory CD4 Treg',
   'KLRB1+ memory CD4 Treg',
   'Memory CD8 Treg',
   'GZMK+ memory CD4 Treg',
   'KLRB1+ memory CD8 Treg'],
  'gdT': ['GZMK+ Vd2 gdT',
   'GZMB+ Vd2 gdT',
   'Naive Vd1 gdT',
   'KLRF1+ effector Vd1 gdT',
   'SOX4+ Vd1 gdT',
   'KLRF1- effector Vd1 gdT']}}
# Preparing data for CSV
csv_data = []
for parent_type, subtypes in nested_dict.items():
    for subtype, cell_types in subtypes.items():
        for cell_type in cell_types:
            csv_data.append((parent_type, subtype, cell_type))

# Path for the CSV file
csv_file_path = 'cell_types.csv'

# Writing data to CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Parent Type", "Subtype", "Cell Type"])  # Header row
    for row in csv_data:
        writer.writerow(row)
